In [ ]:
 # cnn model trained on CIFAR-10 dataset

In [19]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset,DataLoader

import torch.nn.functional as F

In [8]:
# transformations
composed = transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                        ])

In [9]:
# CIFAR-10 dataset
train_data = torchvision.datasets.CIFAR10(root='./data',train=True,
                                          download=True,transform=composed)

test_data = torchvision.datasets.CIFAR10(root='./data',train=False,
                                         download=True,transform=composed)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [17]:
# hyperparameters
batch_size = 4
num_epochs = 10
learning_rate = 0.01

# device
device = None
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


In [12]:
# DataLoaders

train_loader = DataLoader(dataset=train_data,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=2)

test_loader = DataLoader(dataset=test_data,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=2)

In [15]:
len(train_loader),len(test_loader)

(12500, 2500)

In [18]:
# label classes
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

num_classes = len(classes)
print(num_classes)

10


In [20]:
# model class
class ConvNet(nn.Module):
    def __init__(self,num_classes):
        super(ConvNet,self).__init__()
        
        # cnn layers
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5)
        
        #fc layers
        self.linear1 = nn.Linear(in_features=16*5*5,out_features=120)
        self.linear2 = nn.Linear(in_features=120,out_features=84)
        self.linear3 = nn.Linear(in_features=84,out_features=num_classes)

    def forward(self,x):
        x1 = self.pool(F.relu(self.conv1(x)))
        x2 = self.pool(F.relu(self.conv2(x1)))
        x2_flat = x2.view(-1,16*5*5) # Flatten
        x3 = F.relu(self.linear1(x2_flat))
        x4 = F.relu(self.linear2(x3))
        x5 = F.relu(self.linear3(x4))
        return x5

In [22]:
# model defining
model = ConvNet(num_classes)
model = model.to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [23]:
n_total_steps = len(train_loader)

# training loop
for epoch in range(num_epochs):
    for ii,(images,labels) in enumerate(train_loader):
        # original shape = [4,3,32,32] : 4 := batch_size, 3 := RGB, dims =32x32
        # input_layer = 3 input_channels, 6 output_channels, 5 kernel_size
        images = images.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(images)
        loss = criterion(outputs,labels)

        # backward and updates
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if (ii+1)%2000 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{ii+1}/{n_total_steps}], Loss = {loss.item():.6f}')

    print('----------------------------------------')

Epoch [1/10], Step [2000/12500], Loss = 2.596829
Epoch [1/10], Step [4000/12500], Loss = 1.889913
Epoch [1/10], Step [6000/12500], Loss = 2.193358
Epoch [1/10], Step [8000/12500], Loss = 2.971833
Epoch [1/10], Step [10000/12500], Loss = 3.085864
Epoch [1/10], Step [12000/12500], Loss = 1.033718
----------------------------------------
Epoch [2/10], Step [2000/12500], Loss = 1.552075
Epoch [2/10], Step [4000/12500], Loss = 1.272144
Epoch [2/10], Step [6000/12500], Loss = 1.682622
Epoch [2/10], Step [8000/12500], Loss = 0.977196
Epoch [2/10], Step [10000/12500], Loss = 0.737638
Epoch [2/10], Step [12000/12500], Loss = 1.521685
----------------------------------------
Epoch [3/10], Step [2000/12500], Loss = 2.104290
Epoch [3/10], Step [4000/12500], Loss = 1.776286
Epoch [3/10], Step [6000/12500], Loss = 0.531293
Epoch [3/10], Step [8000/12500], Loss = 0.830762
Epoch [3/10], Step [10000/12500], Loss = 1.024097
Epoch [3/10], Step [12000/12500], Loss = 0.833280
------------------------------

In [24]:
# evaluating model

with torch.no_grad():
    # for entire test set
    n_correct = 0
    n_samples = 0

    # for each class label
    n_class_correct = [0 for i in range(num_classes)]
    n_class_samples = [0 for i in range(num_classes)]

    for images,labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        _,preds = torch.max(outputs,1)
        n_samples += labels.size(0)
        n_correct += (preds == labels).sum().item()

        # iterating over the batch 
        for i in range(batch_size):
            label = labels[i]
            pred = preds[i]
            n_class_samples[label] +=1
            if(label == pred):
                n_class_correct[label] += 1
            
    accuracy = n_correct/float(n_samples)
    print(f'Accuracy of model on test set = {accuracy:.4f}')
    
    print('-------------------------------------------')
    
    # printing accuracy per class
    for i in range(num_classes):
        accuracy = n_class_correct[i]/float(n_class_samples[i])
        print(f'Accuracy of {classes[i]} : {accuracy}')

Accuracy of model on test set = 0.6430
-------------------------------------------
Accuracy of plane : 0.711
Accuracy of car : 0.846
Accuracy of bird : 0.43
Accuracy of cat : 0.524
Accuracy of deer : 0.644
Accuracy of dog : 0.473
Accuracy of frog : 0.7
Accuracy of horse : 0.72
Accuracy of ship : 0.738
Accuracy of truck : 0.644


In [29]:
# saving model
MODEL_PATH = 'model.pth'
torch.save(model.state_dict(),MODEL_PATH)

In [30]:
# printing parameters of the model

for param in model.parameters():
    print(param.shape)
    print(str(param))

torch.Size([6, 3, 5, 5])
Parameter containing:
tensor([[[[ 3.3287e-01,  6.0073e-01,  6.1119e-01,  5.1291e-01,  2.4391e-01],
          [ 2.1776e-01,  1.6950e-01,  3.6638e-01,  3.6824e-02,  1.2162e-01],
          [-6.8717e-02, -2.3781e-01, -2.7297e-02, -1.3946e-01,  3.2651e-01],
          [ 1.1522e-01, -2.6747e-01, -3.9195e-01, -1.4185e-01,  6.0015e-01],
          [-3.9254e-01, -6.6540e-01, -5.8051e-01, -2.2269e-01,  2.9562e-01]],

         [[-6.4331e-01, -4.6063e-01, -5.9473e-01, -5.8637e-01, -5.5769e-01],
          [-3.1318e-01, -3.1919e-01, -3.7058e-01, -3.4217e-01, -1.4683e-01],
          [-6.9673e-02, -4.7429e-01, -5.1814e-01, -1.1666e-01,  3.0737e-01],
          [ 1.6787e-01, -5.8573e-01, -5.1279e-01, -4.0897e-02,  5.0933e-01],
          [ 6.7994e-02, -3.9576e-01, -1.4911e-01,  1.3233e-01,  5.5746e-01]],

         [[-8.1871e-02,  1.9333e-01,  2.1725e-01,  1.1576e-01, -3.5592e-01],
          [ 2.1050e-01,  3.4375e-02,  1.5371e-01,  1.9345e-01, -1.1610e-01],
          [ 7.8045e-02, -